In [37]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [38]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-04 04:46:28--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.74MB/s    in 0.2s    

2020-08-04 04:46:28 (4.74 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [39]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [40]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_game_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_game_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [41]:
# number of records in the dataset
video_game_df.count()

1785997

In [42]:
video_game_df.describe().show()

+-------+-----------+--------------------+--------------+-------------------+-------------------+---------------------+----------------+------------------+------------------+-----------------+-------+-----------------+---------------+--------------------+
|summary|marketplace|         customer_id|     review_id|         product_id|     product_parent|        product_title|product_category|       star_rating|     helpful_votes|      total_votes|   vine|verified_purchase|review_headline|         review_body|
+-------+-----------+--------------------+--------------+-------------------+-------------------+---------------------+----------------+------------------+------------------+-----------------+-------+-----------------+---------------+--------------------+
|  count|    1785997|             1785997|       1785997|            1785997|            1785997|              1785997|         1785997|           1785997|           1785997|          1785997|1785997|          1785997|        178599

In [43]:
review_id_df = video_game_df.select(['review_id','customer_id','product_id','product_parent', 'review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809|2015-08-31 00:00:00|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175|2015-08-31 00:00:00|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668|2015-08-31 00:00:00|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350|2015-08-31 00:00:00|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511|2015-08-31 00:00:00|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615|2015-08-31 00:00:00|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603|2015-08-31 00:00:00|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517|2015-08-31 00:00:00|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890|2015-08-31 00:00:00|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651|2015-08-31 00:00:00|
|R1VTIA3JTYBY02|     1065

In [44]:
products_df = video_game_df.select(['product_id','product_title']).distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [45]:
group_df = video_game_df.groupBy("customer_id").count()
# withColumnRenamed does not modify current dataFrame, instead it returns the modified version
customers_df = group_df.withColumnRenamed('count','customer_count')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [46]:
vine_df = video_game_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

In [47]:
# Configure settings for RDS data-bootcamp.crrqs5jt3ck3.us-east-1.rds.amazonaws.com
mode = "append"
jdbc_url="jdbc:postgresql://data-bootcamp.crrqs5jt3ck3.us-east-1.rds.amazonaws.com:5432/bigdatahomework"
config = {"user":"xxxxx", 
          "password": "xxxxxxxx", 
          "driver":"org.postgresql.Driver"}


# write dataframe to tabels

In [48]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)